---
---

**Quantitative Finance and Financial Risk**\
**Project Title: Pricing an American Straddle**

Athens University of Economics and Business\
School of Information Sciences and Technology\
M.Sc. in Data Science, Full Time\
Spring Trimester, June, 2023

Agiopoulos S. Kaptsikas (f3352208, akaptsikas@aueb.gr)

---
---

# <u><b>Contents</b></u>
1. [Description](#1)
2. [Pricing an American-style At-The-Money Straddle using Black & Scholes Option Pricing Model](#2)
3. [Pricing a European-style At-The-Money Straddle using Black & Scholes Option Pricing Model and Monte-Carlo Simulation](#3)

---

In [1]:
import numpy as np
from typing import Tuple
from scipy.stats import norm

# <a id='1'>1. <b>Project Description</b></a>

<p align="justify">Assume a stock the process of which follows a Geometric Brownian Motion. The current stock price S(0) is equal to $20, with an annual volatility of returns of 35%. If the risk-free rate is equal to 3% and the dividend yield is equal to 1.5%, find the price of an American-style At-The-Money Straddle (Long Put and Long Call), with an expiry of 6 months. Find also the stock (delta) and volatility (vega) sensitivities, with the help of your function. Assuming the same stock price process, but in a Monte-Carlo simulation setup, find the price of the equivalent European-style straddle.</p>

* Given the description, above, we can extract useful information that will be used later. That is,

    * **Stock Price**: $S_{0}=\$20$
    * **Volatility**: $\sigma=35\%$
    * **Risk-free Rate**: $r=3\%$
    * **Divident Yield**: $\delta=1.5\%$
    * **Time to Expiration**: $T=0.5$
    * **Strike Price**: $K=\$20$

In [2]:
parameters = {"S": 20, "K": 20, "sigma": .35, "r": .03, "delta": .015, "T": .5}

* Two main questions that need to be addressed, can be identified. In particular,
    1. What is the price of the American-style At-The-Money Straddle?
        * This question is answered in the first part of the notebook.
        * The mathematical model considered for answering, both this, and the second question, is the Black & Scholes option pricing model.
    2. What is the price of the European-style At-The-Money Straddle?
        * This question is answered in the second part of the notebook.
        * As prompted, a Monte-Carlo simulation framework is configured to answer this question, again, assuming the Black & Scholes model.

---

# <a id="2">2. <b>Pricing an American-style At-The-Money Straddle using Black & Scholes Option Pricing Model</b></a>

* The straddle's price is equal to the price of both a long call and long put option, i.e.,

    > $\textbf{P}(S,K,\sigma,r,T,\delta)=C(S,K,\sigma,r,T,\delta)+P(S,K,\sigma,r,T,\delta) \textbf{(1)}$
    >
    > where:
    >
    > $\textbf{P}$: Price of the American-style Straddle\
    > $C(S,K,\sigma,r,T,\delta)$: Price of the American-style Long Call Option\
    > $P(S,K,\sigma,r,T,\delta)$: Price of the American-style Long Put Option
    
* The prices of the American-style Long Call and Long Put Options, can be found using the Black & Scholes model:

    > $C(S,K,\sigma,r,T,\delta) = S e^{- \delta T} N(d_{1}) - K e^{- r T} N(d_{2}) \textbf{(2)}$\
    > $P(S,K,\sigma,r,T,\delta) = K e^{- r T} N(- d_{2}) - S e^{- \delta T} N(- d_{1}) \textbf{(3)}$
    >
    > where:
    >
    > $N(\cdot)$: Cumulative standard normal distribution function $ \textbf{(4)}$
    >
    > $d_{1} = \dfrac{ ln\left(\dfrac{S}{K}\right) + \left(r - \delta + \dfrac{\sigma^{2}}{2} T \right) }{\sigma \sqrt{T}}, \ d_{2} = d_{1} - \sigma \sqrt{T} \textbf{(5)}$

* The stock (delta) and volatility (vega) sensitivities of the American-style Straddle are the following:

    > $ \Delta = N \left( d_{1} \right) \textbf{(6)}$\
    > $v = S N ^ {\prime} \left( d_{1} \right) \sqrt{T} \textbf{(7)}$

* In the code cell, below, we utilize Eqs. $\textbf{(1)}$ through $\textbf{(5)}$, and create a Python function that calculates the price, $\textbf{P}$, of an American-style straddle, which is associated with the following set of parameters: $\{S,K,T,r,\sigma,\delta\}$.

In [3]:
class black_and_scholes:
    """class created for pricing an american straddle"""

    def __init__(self, **kwargs) -> None:
        """initialization. configuring parameters."""

        self.parameters                 = kwargs                                # save given parameters

        self.S                          = kwargs["S"]                           # current stock price
        self.K                          = kwargs["K"]                           # strike price
        self.T                          = kwargs["T"]                           # expiration
        self.r                          = kwargs["r"]                           # risk free rate
        self.sigma                      = kwargs["sigma"]                       # volatility
        self.delta                      = kwargs["delta"]                       # divident yield

        self.d1, self.d2                = self.calculate_d1_and_d2()            # d1 and d2
        self.Nd1, self.Nd2              = norm.cdf(self.d1), norm.cdf(self.d2)  # N(d1) and N(d2): standard normal cumulative density function
        self.Nminusd1, self.Nminusd2    = (1 - self.Nd1), (1 - self.Nd2)        # N(-d1) and N(-d2)
        self.Nprimed1                   = norm.pdf(self.d1)                     # N'(d1): standard normal probability density function

        self.Delta                      = self.Nd1                              # stock sensitivity
        self.vega                       = self.S * self.Nprimed1                # volatility sensitivity

        self.C                          = self.calculate_call_option_price()    # call option price
        self.P                          = self.calculate_put_option_price()     # put option price

        self.straddle_price             = self.C + self.P                       # straddle price

        print(f"""
      
                SUMMARY OF RESULTS
                    
                at-the-money, american-style, straddle
                    
                    I.      parameters:
                    
                            1.  S                   = {self.S},
                            2.  K                   = {self.K},
                            3.  sigma               = {self.sigma},
                            4.  r                   = {self.r},
                            5.  delta               = {self.delta},
                            6.  T                   = {self.T}

                    II.     prices:
                    
                            1.  straddle            = {self.straddle_price}
                            2.  call                = {self.C}
                            3.  put                 = {self.P}

                    III.    sensitivities:

                            1.  stock (delta)       = {self.Delta}
                            2.  volatility (vega)   = {self.vega}

            """)

    def calculate_d1_and_d2(self, N = False) -> Tuple[float, float]:
        """method for calculating d1 and d2. see eq. (5), in section 2, for the details of calculation."""

        d1 = (np.log(self.S / self.K) + (self.r - self.delta + (((self.sigma ** 2) *.5) * self.T))) / (self.sigma * np.sqrt(self.T))
        d2 = d1 - (self.sigma * np.sqrt(self.T))

        return d1, d2
        
    def calculate_call_option_price(self) -> float:
        """method for calculating the price of the call option. see eq. (2), in section 2, for the details of calculation."""

        return (self.S * self.Nd1) - (self.K * np.exp(- self.r * self.T) * self.Nd2)
    
    def calculate_put_option_price(self) -> float:
        """method for calculating the price of the put option. see eq. (3), in section 2, for the details of calculation."""

        return (self.K * np.exp(- self.r * self.T) * self.Nminusd2) - (self.S * self.Nminusd1)

In [4]:
american_straddle = black_and_scholes(**parameters)


      
                SUMMARY OF RESULTS
                    
                at-the-money, american-style, straddle
                    
                    I.      parameters:
                    
                            1.  S                   = 20,
                            2.  K                   = 20,
                            3.  sigma               = 0.35,
                            4.  r                   = 0.03,
                            5.  delta               = 0.015,
                            6.  T                   = 0.5

                    II.     prices:
                    
                            1.  straddle            = 3.917089728206971
                            2.  call                = 2.1074254680728597
                            3.  put                 = 1.8096642601341113

                    III.    sensitivities:

                            1.  stock (delta)       = 0.5731316678172701
                            2.  volatility (vega) 

---

# <a id="3">3. <b>Pricing a European-style At-The-Money Straddle using Black & Scholes Option Pricing Model and Monte-Carlo Simulation</b></a>

* When considering the straddle's pricing under a Monte-Carlo simulation scheme, the price is determined as follows:

    > $\textbf{P}_{0}(S,K,\sigma,r,T,\delta) = PV \left( C_{Payoff} \right) + PV \left( P_{Payoff} \right) \textbf{(1)}$
    >
    > where:
    >
    > $\textbf{P}$: Price of the European-style Straddle\
    > $C_{Payoff}$: Payoff of the European-style Long Call Option\
    > $P_{Payoff}$: Payoff of the European-style Long Put Option
    
* The payoffs of the European-style Long Call and Long Put options can be found, as follows:

    > $C_{Payoff} = max \left( 0, S_{T} - K \right) \textbf{(2)}$\
    > $P_{Payoff} = max \left( 0, K - S_{T} \right) \textbf{(3)}$
    >
    > where:
    >
    > $S_{T}$: Stock price at expiration

* The stock's price is a random process that follows a Geometric Brownian Motion, and can be analytically calculated, using the formula:

    > $ S_{T} = S_{0} e^{ \left( \left( r - \delta - \dfrac{\sigma^{2}}{2} \right)T + \sigma \sqrt{T} Z_{T} \right) } \textbf{(6)}\ $
    >
    > where:
    >
    > $Z_{T} \sim N \left( 0, 1 \right) \textbf{(7)}$: Random variable following the standard normal distribution

* Given the stochastic nature of the stock's price, we can derive an estimation of the straddle's price, using a risk-neutral pricing:

    > $\textbf{P}_{0}(S,K,\sigma,r,T,\delta) = \mathbb{E} \left[ \textbf{P} \right] = \mathbb{E} \left[ PV \left( C_{Payoff} \right) + PV \left( P_{Payoff} \right) \right] = \mathbb{E} \left[ PV \left( max \left( 0, S_{T} - K \right) \right) + PV \left( max \left( 0, K - S_{T} \right) \right) \right] \textbf{(7)}$ 
    >

In [5]:
class monte_carlo_simulation:
    """class created for pricing a european-style straddle."""

    def __init__(self, N: int, **kwargs) -> None:
        """initialization. configuring parameters."""

        self.parameters                 = kwargs                                                                    # save given parameters

        self.S0                         = kwargs["S"]                                                               # current stock price
        self.K                          = kwargs["K"]                                                               # strike price
        self.T                          = kwargs["T"]                                                               # expiration
        self.r                          = kwargs["r"]                                                               # risk free rate
        self.sigma                      = kwargs["sigma"]                                                           # volatility
        self.delta                      = kwargs["delta"]                                                           # divident yield
        self.N                          = N                                                                         # sample size (simulation)

        self.Z                          = self.generate_random_numbers(N = self.N)                                  # random variable following the standard normal distribution
        self.ST                         = self.calculate_stock_prices_at_expiration(Z = self.Z)                     # stock price at expiration
        self.straddle_payoff            = self.calculate_straddle_payoff(ST = self.ST)                              # straddle's average payoff, given random stock prices at expiration
        self.straddle_price             = self.calculate_straddle_price(straddle_payoff = self.straddle_payoff)     # straddle's average price is the present value of the straddle's average payoff

        print(f"""

                SUMMARY OF RESULTS
            
                at-the-money, european-style, straddle
            
                    I.      parameters:
            
                            1.  S                   = {self.S0},
                            2.  K                   = {self.K},
                            3.  sigma               = {self.sigma},
                            4.  r                   = {self.r},
                            5.  delta               = {self.delta},
                            6.  T                   = {self.T},
                            7.  N                   = {self.N}

                    II.     simulated prices:
            
                            1.  straddle            = {self.straddle_price}

                """)

    def generate_random_numbers(self, N: int) -> np.ndarray:
        """generate idependent and identically distributed random numbers from the standard normal distribution"""
        return np.expand_dims(norm.rvs(size = N), axis = -1)
    
    def calculate_stock_prices_at_expiration(self, Z: np.ndarray) -> np.ndarray:
        """calculate stock prices at expiration. see eq. (6), in section 3, for the details of calculation."""
        return self.S0 * np.exp(((self.r - self.delta - (self.sigma * .5)) * self.T) + (self.sigma * np.sqrt(self.T) * Z))

    def calculate_straddle_payoff(self, ST: np.ndarray) -> float:
        """calculate straddle's payoff. see eq. (7), in section 3, for the details of calculation."""

        call_payoff = np.where(ST - self.K > 0, ST - self.K, 0)
        put_payoff = np.where(self.K - ST > 0, self.K - ST, 0)
        straddle_payoff = np.average(call_payoff + put_payoff)
        
        return float(straddle_payoff)
    
    def calculate_straddle_price(self, straddle_payoff: float) -> float:
        """calculate straddle's price, see eq. (7), in section 3, for the details of calculation."""
        return straddle_payoff * np.exp(- self.r * self.T)

In [6]:
european_straddle = monte_carlo_simulation(N = 10000, **parameters)



                SUMMARY OF RESULTS
            
                at-the-money, european-style, straddle
            
                    I.      parameters:
            
                            1.  S                   = 20,
                            2.  K                   = 20,
                            3.  sigma               = 0.35,
                            4.  r                   = 0.03,
                            5.  delta               = 0.015,
                            6.  T                   = 0.5,
                            7.  N                   = 10000

                    II.     simulated prices:
            
                            1.  straddle            = 3.8647363069337612

                


---
---
---